In [26]:
# importing necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## NPO DATA

In [28]:
# load in scraped dataframe
df_npo = pd.read_csv('/content/drive/MyDrive/PPM/npo_fv.csv', sep=',')
df_npo

,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,genre1_weight,genre2_weight,political_bias_bc,polarization_score,inverted_polarization_score,Episode
0,0,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,"Met de groei van sociale media, zoals Snapchat...",https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,1
1,1,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Jurre en Dzifa duiken dieper in het brein; ze ...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,2
2,2,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Dzifa en Jurre onderzoeken of de dickpic missc...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,3
3,3,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Dzifa en Jurre kijken naar de toekomst; waar g...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,4
4,4,30 en nooit meer...,https://assets-start.npo.nl/resources/2023/09/...,Documentaire,Human Interest,PowNed,30 en nooit meer ongelukkig,In Wales gaat Mara op zoek naar geluk bij een ...,https://assets-start.npo.nl/resources/2023/09/...,30 en nooit meer... Documentaire Human Interes...,1.25,1.25,2.0,3.125,0.6875,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,1947,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,Geen wolkje aan de lucht,"Nederland staat er economisch goed voor, de Pv...",https://images.poms.omroep.nl/image/1682902,Het jaar van Fortuyn Serie Drama Geen wolkje a...,0.50,0.00,-0.5,-0.000,1.0000,1
1948,1948,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,Ik zeg wat ik denk,Melkert wil stelling nemen over de groeiende m...,https://images.poms.omroep.nl/image/1682903,Het jaar van Fortuyn Serie Drama Ik zeg wat ik...,0.50,0.00,-0.5,-0.000,1.0000,2
1949,1949,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,De gewone man,Fortuyn besluit na de breuk met Leefbaar Neder...,https://images.poms.omroep.nl/image/1682904,Het jaar van Fortuyn Serie Drama De gewone man...,0.50,0.00,-0.5,-0.000,1.0000,3
1950,1950,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,There is no business like showbusiness,Als Fortuyn een taart in zijn gezicht krijgt v...,https://images.poms.omroep.nl/image/1682905,Het jaar van Fortuyn Serie Drama There is no b...,0.50,0.00,-0.5,-0.000,1.0000,4


In [29]:
# create new variable that includes all relevant information for tf-idf similarity calculation
df_npo['Content'] = df_npo['Serie']
df_npo.fillna('no_genre', inplace=True)

for index, row in df_npo.iterrows():
    if row['Serie'] != row['Title']:
        df_npo.at[index, 'Content'] += ' ' + row['Genre_1'] + ' ' + row['Genre_2'] + ' ' + row['Title'] + ' ' + row['Description']
    else:
        df_npo.at[index, 'Content'] += ' ' + row['Genre_1'] + ' ' + row['Genre_2'] + ' ' + row['Description']

df_npo = df_npo.drop_duplicates()
df_npo

,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,genre1_weight,genre2_weight,political_bias_bc,polarization_score,inverted_polarization_score,Episode
0,0,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,"Met de groei van sociale media, zoals Snapchat...",https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.0,-3.5,-2.625,0.7375,1
1,1,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Jurre en Dzifa duiken dieper in het brein; ze ...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.0,-3.5,-2.625,0.7375,2
2,2,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Dzifa en Jurre onderzoeken of de dickpic missc...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.0,-3.5,-2.625,0.7375,3
3,3,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Dzifa en Jurre kijken naar de toekomst; waar g...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.0,-3.5,-2.625,0.7375,4
4,4,30 en nooit meer...,https://assets-start.npo.nl/resources/2023/09/...,Documentaire,Human Interest,PowNed,30 en nooit meer ongelukkig,In Wales gaat Mara op zoek naar geluk bij een ...,https://assets-start.npo.nl/resources/2023/09/...,30 en nooit meer... Documentaire Human Interes...,1.25,1.25,2.0,3.125,0.6875,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,1947,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,Geen wolkje aan de lucht,"Nederland staat er economisch goed voor, de Pv...",https://images.poms.omroep.nl/image/1682902,Het jaar van Fortuyn Serie Drama Geen wolkje a...,0.50,0.0,-0.5,-0.0,1.0,1
1948,1948,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,Ik zeg wat ik denk,Melkert wil stelling nemen over de groeiende m...,https://images.poms.omroep.nl/image/1682903,Het jaar van Fortuyn Serie Drama Ik zeg wat ik...,0.50,0.0,-0.5,-0.0,1.0,2
1949,1949,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,De gewone man,Fortuyn besluit na de breuk met Leefbaar Neder...,https://images.poms.omroep.nl/image/1682904,Het jaar van Fortuyn Serie Drama De gewone man...,0.50,0.0,-0.5,-0.0,1.0,3
1950,1950,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,There is no business like showbusiness,Als Fortuyn een taart in zijn gezicht krijgt v...,https://images.poms.omroep.nl/image/1682905,Het jaar van Fortuyn Serie Drama There is no b...,0.50,0.0,-0.5,-0.0,1.0,4


## POLARIZATION

In [30]:
# political bias per broadcaster
bc_value = {'AVROTROS': -0.5, 'BNNVARA': -3.5, 'VPRO': -2, 'KRO-NCRV': -1, 'NTR': -1.5, 'HUMAN': -2, 'Omroep Zwart': -5,
            'NOS': 0, 'MAX': 1, 'PowNed': 2, 'EO': 2.5, 'WNL': 3.5, 'Omroepvereniging Ongehoord Nederland': 5}
# polarization weight per genre
g1_value = {'Sport': 0, 'Jeugd': 0, 'Film': 0.25, 'Serie': 0.5, 'Amusement': 1,
             'Human Interest': 0.75, 'Informatief': 1.25, 'Documentaire': 1.25}
g2_value = {'Drama': 0, 'Serie': 0, 'Komisch': 0, 'Sport - informatie': 0, 'Liefde': 0, 'Spanning': 0, 'Kunst/cultuur': 0.5, 'Muziek': 0,
            'Sport': 0, 'Spel/quiz': 0, 'Horror': 0, 'Science Fiction': 0, 'Wetenschap': 1, 'Natuur': 1, 'Reality TV': 1, 'Gezondheid/opvoeding': 1,
            'Biografie': 1, 'Familie': 1, 'no_genre': 1, 'Religieus': 1.25, 'Consumenten informatie': 1.25, 'Human Interest': 1.25, 'Reizen': 1.25,
            'Geschiedenis': 1.25, 'Levenswijze': 1.25, 'Reportage': 1.25, '(sub)culturen': 1.5, 'Komisch/Satire': 1.5, 'Onderzoeksjournalistiek': 1.5,
            'Cabaret': 1.5, 'Nieuws/actualiteiten': 1.5, 'Politiek': 1.75, 'Maatschappelijk Debat': 1.75, 'Interviewprogramma/hosted show': 1.75}
# add political bias and genre weights to recommendations
df_npo['political_bias_bc'] = df_npo['Broadcaster'].map(bc_value)
df_npo['genre1_weight'] = df_npo['Genre_1'].map(g1_value)
df_npo['genre2_weight'] = df_npo['Genre_2'].map(g2_value)
# calculate polarization score per recommendation
df_npo['polarization_score'] = (df_npo['genre1_weight'] * df_npo['genre2_weight']) * df_npo['political_bias_bc']

In [31]:
# create inverted normalized polarization score
df_npo['inverted_polarization_score'] = 1 - (abs(df_npo['polarization_score']) / 10)

In [32]:
df_npo

,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,genre1_weight,genre2_weight,political_bias_bc,polarization_score,inverted_polarization_score,Episode
0,0,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,"Met de groei van sociale media, zoals Snapchat...",https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,1
1,1,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Jurre en Dzifa duiken dieper in het brein; ze ...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,2
2,2,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Dzifa en Jurre onderzoeken of de dickpic missc...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,3
3,3,Dickpics: de keiharde wetenschap,https://assets-start.npo.nl/resources/2023/11/...,Human Interest,Gezondheid/opvoeding,BNNVARA,Dickpics: de keiharde wetenschap,Dzifa en Jurre kijken naar de toekomst; waar g...,https://assets-start.npo.nl/resources/2023/11/...,Dickpics: de keiharde wetenschap Human Interes...,0.75,1.00,-3.5,-2.625,0.7375,4
4,4,30 en nooit meer...,https://assets-start.npo.nl/resources/2023/09/...,Documentaire,Human Interest,PowNed,30 en nooit meer ongelukkig,In Wales gaat Mara op zoek naar geluk bij een ...,https://assets-start.npo.nl/resources/2023/09/...,30 en nooit meer... Documentaire Human Interes...,1.25,1.25,2.0,3.125,0.6875,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,1947,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,Geen wolkje aan de lucht,"Nederland staat er economisch goed voor, de Pv...",https://images.poms.omroep.nl/image/1682902,Het jaar van Fortuyn Serie Drama Geen wolkje a...,0.50,0.00,-0.5,-0.000,1.0000,1
1948,1948,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,Ik zeg wat ik denk,Melkert wil stelling nemen over de groeiende m...,https://images.poms.omroep.nl/image/1682903,Het jaar van Fortuyn Serie Drama Ik zeg wat ik...,0.50,0.00,-0.5,-0.000,1.0000,2
1949,1949,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,De gewone man,Fortuyn besluit na de breuk met Leefbaar Neder...,https://images.poms.omroep.nl/image/1682904,Het jaar van Fortuyn Serie Drama De gewone man...,0.50,0.00,-0.5,-0.000,1.0000,3
1950,1950,Het jaar van Fortuyn,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,AVROTROS,There is no business like showbusiness,Als Fortuyn een taart in zijn gezicht krijgt v...,https://images.poms.omroep.nl/image/1682905,Het jaar van Fortuyn Serie Drama There is no b...,0.50,0.00,-0.5,-0.000,1.0000,4


## COSINE SIMILARITY

In [33]:
# load dutch stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
dutch_stopwords = stopwords.words('dutch')

# vectorize articles based on their title
vectorizer = TfidfVectorizer(strip_accents='unicode', lowercase=True, stop_words=dutch_stopwords)
articles_vectors = vectorizer.fit_transform(df_npo['Content'])

# calculate cosine similarity between article vectors and create similarity matrix
sim_matrix = cosine_similarity(articles_vectors, articles_vectors)
sim_df = pd.DataFrame(sim_matrix, index=df_npo.index, columns=df_npo.index)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## PERSONAS

In [34]:
# create personas for user dataset
personas = {'Ambitieuze Jongeren': {
  'Gezondheid/opvoeding': 0.01587301587301588,
  'Human Interest': 0.01587301587301588,
  'Nieuws/actualiteiten': 0.01587301587301588,
  'Natuur': 0.01587301587301588,
  'Reizen': 0.01587301587301588,
  'Geschiedenis': 0.01587301587301588,
  'Politiek': 0.00793650793650794,
  'Documentaire': 0.01587301587301588,
  'Drama': 0.04761904761904762,
  'Levenswijze': 0.03174603174603176,
  'Consumenten informatie': 0.00793650793650794,
  'Familie': 0.03174603174603176,
  'Biografie': 0.01587301587301588,
  'Informatief': 0.01587301587301588,
  'Reality TV': 0.05555555555555556,
  'Onderzoeksjournalistiek': 0.00793650793650794,
  'Amusement': 0.04761904761904762,
  'Komisch/Satire': 0.05555555555555556,
  'Science Fiction': 0.04761904761904762,
  'Spanning': 0.04761904761904762,
  'Horror': 0.04761904761904762,
  'Muziek': 0.04761904761904762,
  'Reportage': 0.01587301587301588,
  '(sub)culturen': 0.00793650793650794,
  'Sport': 0.04761904761904762,
  'Liefde': 0.04761904761904762,
  'Spel/quiz': 0.04761904761904762,
  'Maatschappelijk Debat': 0.00793650793650794,
  'Serie': 0.03174603174603176,
  'Kunst/cultuur': 0.01587301587301588,
  'Cabaret': 0.03174603174603176,
  'Sport - informatie': 0.01587301587301588,
  'Wetenschap': 0.03174603174603176,
  'Interviewprogramma/hosted show': 0.04761904761904762,
  'Religieus': 0.01587301587301588,
  'Komisch': 0.00793650793650794,
  'Broadcasters':{
        'AVROTROS': 0.15,
        'PowNed': 0.1,
        'MAX': 0.025,
        'WNL': 0.025,
        'VPRO': 0.15,
        'HUMAN': 0.05,
        'BNNVARA': 0.15,
        'KRO-NCRV': 0.1,
        'EO': 0.05,
        'Omroepvereniging Ongehoord Nederland': 0.05,
        'ZWART': 0.05,
        'NOS': 0.05,
        'NTR': 0.05
    }
  },


 'Avontuurlijke Stadsbewoners': {
  'Gezondheid/opvoeding': 0.022099447513812157,
  'Human Interest': 0.03314917127071824,
  'Nieuws/actualiteiten': 0.03867403314917127,
  'Natuur': 0.03867403314917127,
  'Reizen': 0.03867403314917127,
  'Geschiedenis': 0.03867403314917127,
  'Politiek': 0.03867403314917127,
  'Documentaire': 0.03314917127071824,
  'Drama': 0.022099447513812157,
  'Levenswijze': 0.03314917127071824,
  'Consumenten informatie': 0.011049723756906079,
  'Familie': 0.022099447513812157,
  'Biografie': 0.022099447513812157,
  'Informatief': 0.022099447513812157,
  'Reality TV': 0.022099447513812157,
  'Onderzoeksjournalistiek': 0.03314917127071824,
  'Amusement': 0.022099447513812157,
  'Komisch/Satire': 0.022099447513812157,
  'Science Fiction': 0.022099447513812157,
  'Spanning': 0.022099447513812157,
  'Horror': 0.022099447513812157,
  'Muziek': 0.03314917127071824,
  'Reportage': 0.03314917127071824,
  '(sub)culturen': 0.03314917127071824,
  'Sport': 0.03314917127071824,
  'Liefde': 0.022099447513812157,
  'Spel/quiz': 0.011049723756906079,
  'Maatschappelijk Debat': 0.03314917127071824,
  'Serie': 0.022099447513812157,
  'Kunst/cultuur': 0.03314917127071824,
  'Cabaret': 0.03314917127071824,
  'Sport - informatie': 0.03314917127071824,
  'Wetenschap': 0.03314917127071824,
  'Interviewprogramma/hosted show': 0.022099447513812157,
  'Religieus': 0.03314917127071824,
  'Komisch': 0.011049723756906079,
  'Broadcasters':{
        'AVROTROS': 0.1,
        'PowNed': 0.1,
        'MAX': 0.025,
        'WNL': 0.05,
        'VPRO': 0.15,
        'HUMAN': 0.1,
        'BNNVARA': 0.15,
        'KRO-NCRV': 0.05,
        'EO': 0.05,
        'Omroepvereniging Ongehoord Nederland': 0.025,
        'ZWART': 0.05,
        'NOS': 0.1,
        'NTR': 0.05
    }
  },

 'Zelfbewuste Familiemensen': {'Gezondheid/opvoeding': 0.03763440860215054,
  'Human Interest': 0.03225806451612903,
  'Nieuws/actualiteiten': 0.03763440860215054,
  'Natuur': 0.021505376344086027,
  'Reizen': 0.03225806451612903,
  'Geschiedenis': 0.03763440860215054,
  'Politiek': 0.03763440860215054,
  'Documentaire': 0.03225806451612903,
  'Drama': 0.021505376344086027,
  'Levenswijze': 0.03763440860215054,
  'Consumenten informatie': 0.021505376344086027,
  'Familie': 0.03763440860215054,
  'Biografie': 0.03225806451612903,
  'Informatief': 0.03225806451612903,
  'Reality TV': 0.021505376344086027,
  'Onderzoeksjournalistiek': 0.03225806451612903,
  'Amusement': 0.03225806451612903,
  'Komisch/Satire': 0.021505376344086027,
  'Science Fiction': 0.021505376344086027,
  'Spanning': 0.021505376344086027,
  'Horror': 0.021505376344086027,
  'Muziek': 0.03225806451612903,
  'Reportage': 0.021505376344086027,
  '(sub)culturen': 0.021505376344086027,
  'Sport': 0.03225806451612903,
  'Liefde': 0.021505376344086027,
  'Spel/quiz': 0.021505376344086027,
  'Maatschappelijk Debat': 0.03225806451612903,
  'Serie': 0.021505376344086027,
  'Kunst/cultuur': 0.021505376344086027,
  'Cabaret': 0.03225806451612903,
  'Sport - informatie': 0.03225806451612903,
  'Wetenschap': 0.021505376344086027,
  'Interviewprogramma/hosted show': 0.03225806451612903,
  'Religieus': 0.021505376344086027,
  'Komisch': 0.010752688172043013,
  'Broadcasters' : {
        'AVROTROS': 0.1,
        'PowNed': 0.025,
        'MAX': 0.05,
        'WNL': 0.15,
        'VPRO': 0.15,
        'HUMAN': 0.1,
        'BNNVARA': 0.1,
        'KRO-NCRV': 0.05,
        'EO': 0.1,
        'Omroepvereniging Ongehoord Nederland': 0.05,
        'ZWART': 0.025,
        'NOS': 0.05,
        'NTR': 0.05
    }
  },



 'Technische Doeners': {
  'Gezondheid/opvoeding': 0.014184397163120569,
  'Human Interest': 0.014184397163120569,
  'Nieuws/actualiteiten': 0.028368794326241138,
  'Natuur': 0.014184397163120569,
  'Reizen': 0.028368794326241138,
  'Geschiedenis': 0.028368794326241138,
  'Politiek': 0.028368794326241138,
  'Documentaire': 0.028368794326241138,
  'Drama': 0.028368794326241138,
  'Levenswijze': 0.028368794326241138,
  'Consumenten informatie': 0.028368794326241138,
  'Familie': 0.028368794326241138,
  'Biografie': 0.028368794326241138,
  'Informatief': 0.0425531914893617,
  'Reality TV': 0.028368794326241138,
  'Onderzoeksjournalistiek': 0.028368794326241138,
  'Amusement': 0.028368794326241138,
  'Komisch/Satire': 0.028368794326241138,
  'Science Fiction': 0.028368794326241138,
  'Spanning': 0.028368794326241138,
  'Horror': 0.028368794326241138,
  'Muziek': 0.028368794326241138,
  'Reportage': 0.028368794326241138,
  '(sub)culturen': 0.028368794326241138,
  'Sport': 0.04964539007092198,
  'Liefde': 0.014184397163120569,
  'Spel/quiz': 0.028368794326241138,
  'Maatschappelijk Debat': 0.028368794326241138,
  'Serie': 0.028368794326241138,
  'Kunst/cultuur': 0.007092198581560284,
  'Cabaret': 0.028368794326241138,
  'Sport - informatie': 0.04964539007092198,
  'Wetenschap': 0.04964539007092198,
  'Interviewprogramma/hosted show': 0.028368794326241138,
  'Religieus': 0.028368794326241138,
  'Komisch': 0.007092198581560284,
  'Broadcasters':{
        'AVROTROS': 0.1,
        'PowNed': 0.1,
        'MAX': 0.05,
        'WNL': 0.15,
        'VPRO': 0.1,
        'HUMAN': 0.05,
        'BNNVARA': 0.05,
        'KRO-NCRV': 0.05,
        'EO': 0.1,
        'Omroepvereniging Ongehoord Nederland': 0.1,
        'ZWART': 0.0,
        'NOS': 0.1,
        'NTR': 0.05
    }
  },

 'Zorgzame Duizendpoten': {'Gezondheid/opvoeding': 0.049295774647887314,
  'Human Interest': 0.049295774647887314,
  'Nieuws/actualiteiten': 0.028169014084507043,
  'Natuur': 0.028169014084507043,
  'Reizen': 0.049295774647887314,
  'Geschiedenis': 0.014084507042253521,
  'Politiek': 0.014084507042253521,
  'Documentaire': 0.014084507042253521,
  'Drama': 0.028169014084507043,
  'Levenswijze': 0.014084507042253521,
  'Consumenten informatie': 0.014084507042253521,
  'Familie': 0.049295774647887314,
  'Biografie': 0.014084507042253521,
  'Informatief': 0.007042253521126761,
  'Reality TV': 0.049295774647887314,
  'Onderzoeksjournalistiek': 0.014084507042253521,
  'Amusement': 0.04225352112676056,
  'Komisch/Satire': 0.049295774647887314,
  'Science Fiction': 0.014084507042253521,
  'Spanning': 0.028169014084507043,
  'Horror': 0.028169014084507043,
  'Muziek': 0.028169014084507043,
  'Reportage': 0.014084507042253521,
  '(sub)culturen': 0.014084507042253521,
  'Sport': 0.04225352112676056,
  'Liefde': 0.04225352112676056,
  'Spel/quiz': 0.04225352112676056,
  'Maatschappelijk Debat': 0.014084507042253521,
  'Serie': 0.028169014084507043,
  'Kunst/cultuur': 0.014084507042253521,
  'Cabaret': 0.028169014084507043,
  'Sport - informatie': 0.04225352112676056,
  'Wetenschap': 0.007042253521126761,
  'Interviewprogramma/hosted show': 0.04225352112676056,
  'Religieus': 0.028169014084507043,
  'Komisch': 0.014084507042253521,
  'Broadcasters':{
        'AVROTROS': 0.15,
        'PowNed': 0.05,
        'MAX': 0.1,
        'WNL': 0.1,
        'VPRO': 0.1,
        'HUMAN': 0.05,
        'BNNVARA': 0.1,
        'KRO-NCRV': 0.1,
        'EO': 0.1,
        'Omroepvereniging Ongehoord Nederland': 0.05,
        'ZWART': 0.0,
        'NOS': 0.05,
        'NTR': 0.05
    }
  },

 'Authentieke Gelovigen': {
  'Gezondheid/opvoeding': 0.028571428571428574,
  'Human Interest': 0.028571428571428574,
  'Nieuws/actualiteiten': 0.049999999999999996,
  'Natuur': 0.028571428571428574,
  'Reizen': 0.028571428571428574,
  'Geschiedenis': 0.028571428571428574,
  'Politiek': 0.028571428571428574,
  'Documentaire': 0.028571428571428574,
  'Drama': 0.028571428571428574,
  'Levenswijze': 0.028571428571428574,
  'Consumenten informatie': 0.028571428571428574,
  'Familie': 0.028571428571428574,
  'Biografie': 0.028571428571428574,
  'Informatief': 0.028571428571428574,
  'Reality TV': 0.028571428571428574,
  'Onderzoeksjournalistiek': 0.028571428571428574,
  'Amusement': 0.028571428571428574,
  'Komisch/Satire': 0.028571428571428574,
  'Science Fiction': 0.014285714285714287,
  'Spanning': 0.028571428571428574,
  'Horror': 0.014285714285714287,
  'Muziek': 0.028571428571428574,
  'Reportage': 0.028571428571428574,
  '(sub)culturen': 0.028571428571428574,
  'Sport': 0.028571428571428574,
  'Liefde': 0.028571428571428574,
  'Spel/quiz': 0.028571428571428574,
  'Maatschappelijk Debat': 0.028571428571428574,
  'Serie': 0.028571428571428574,
  'Kunst/cultuur': 0.028571428571428574,
  'Cabaret': 0.0071428571428571435,
  'Sport - informatie': 0.028571428571428574,
  'Wetenschap': 0.0071428571428571435,
  'Interviewprogramma/hosted show': 0.028571428571428574,
  'Religieus': 0.028571428571428574,
  'Komisch': 0.049999999999999996,
  'Broadcasters':{
        'AVROTROS': 0.1,
        'PowNed': 0.025,
        'MAX': 0.1,
        'WNL': 0.15,
        'VPRO': 0.1,
        'HUMAN': 0.025,
        'BNNVARA': 0.025,
        'KRO-NCRV': 0.15,
        'EO': 0.15,
        'Omroepvereniging Ongehoord Nederland': 0.1,
        'ZWART': 0.0,
        'NOS': 0.05,
        'NTR': 0.025
    }
  },


 'De Welgestelde Verdiepingszoekers': {
  'Gezondheid/opvoeding': 0.023255813953488375,
  'Human Interest': 0.03488372093023256,
  'Nieuws/actualiteiten': 0.040697674418604654,
  'Natuur': 0.040697674418604654,
  'Reizen': 0.023255813953488375,
  'Geschiedenis': 0.040697674418604654,
  'Politiek': 0.040697674418604654,
  'Documentaire': 0.03488372093023256,
  'Drama': 0.023255813953488375,
  'Levenswijze': 0.03488372093023256,
  'Consumenten informatie': 0.03488372093023256,
  'Familie': 0.03488372093023256,
  'Biografie': 0.03488372093023256,
  'Informatief': 0.03488372093023256,
  'Reality TV': 0.011627906976744188,
  'Onderzoeksjournalistiek': 0.03488372093023256,
  'Amusement': 0.011627906976744188,
  'Komisch/Satire': 0.011627906976744188,
  'Science Fiction': 0.011627906976744188,
  'Spanning': 0.011627906976744188,
  'Horror': 0.011627906976744188,
  'Muziek': 0.023255813953488375,
  'Reportage': 0.03488372093023256,
  '(sub)culturen': 0.03488372093023256,
  'Sport': 0.023255813953488375,
  'Liefde': 0.011627906976744188,
  'Spel/quiz': 0.011627906976744188,
  'Maatschappelijk Debat': 0.03488372093023256,
  'Serie': 0.023255813953488375,
  'Kunst/cultuur': 0.040697674418604654,
  'Cabaret': 0.03488372093023256,
  'Sport - informatie': 0.023255813953488375,
  'Wetenschap': 0.040697674418604654,
  'Interviewprogramma/hosted show': 0.023255813953488375,
  'Religieus': 0.03488372093023256,
  'Komisch': 0.023255813953488375,
  'Broadcasters': {
        'AVROTROS': 0.05,
        'PowNed': 0.025,
        'MAX': 0.05,
        'WNL': 0.05,
        'VPRO': 0.15,
        'HUMAN': 0.15,
        'BNNVARA': 0.15,
        'KRO-NCRV': 0.1,
        'EO': 0.05,
        'Omroepvereniging Ongehoord Nederland': 0.0,
        'ZWART': 0.025,
        'NOS': 0.1,
        'NTR': 0.1
    }
  },

 'Behoedzame Senioren': {'Gezondheid/opvoeding': 0.03125,
  'Human Interest': 0.03125,
  'Nieuws/actualiteiten': 0.05468749999999999,
  'Natuur': 0.03125,
  'Reizen': 0.03125,
  'Geschiedenis': 0.03125,
  'Politiek': 0.015625,
  'Documentaire': 0.03125,
  'Drama': 0.015625,
  'Levenswijze': 0.03125,
  'Consumenten informatie': 0.03125,
  'Familie': 0.05468749999999999,
  'Biografie': 0.03125,
  'Informatief': 0.03125,
  'Reality TV': 0.0078125,
  'Onderzoeksjournalistiek': 0.03125,
  'Amusement': 0.03125,
  'Komisch/Satire': 0.015625,
  'Science Fiction': 0.0078125,
  'Spanning': 0.0078125,
  'Horror': 0.0078125,
  'Muziek': 0.015625,
  'Reportage': 0.03125,
  '(sub)culturen': 0.03125,
  'Sport': 0.03125,
  'Liefde': 0.015625,
  'Spel/quiz': 0.015625,
  'Maatschappelijk Debat': 0.03125,
  'Serie': 0.03125,
  'Kunst/cultuur': 0.03125,
  'Cabaret': 0.03125,
  'Sport - informatie': 0.03125,
  'Wetenschap': 0.03125,
  'Interviewprogramma/hosted show': 0.03125,
  'Religieus': 0.03125,
  'Komisch': 0.04687499999999999,
  'Broadcasters': {
        'AVROTROS': 0.1,
        'PowNed': 0.025,
        'MAX': 0.15,
        'WNL': 0.15,
        'VPRO': 0.05,
        'HUMAN': 0.025,
        'BNNVARA': 0.05,
        'KRO-NCRV': 0.1,
        'EO': 0.1,
        'Omroepvereniging Ongehoord Nederland': 0.1,
        'ZWART': 0.0,
        'NOS': 0.1,
        'NTR': 0.05
    }

  }}

## USERS DATA

In [35]:
import random
import pandas as pd

# Define the number of users for each persona
num_users_per_persona = 125

# Function to synthesize user data based on personas
def synthesize_user_data(persona, preferences, df_npo, user_id_prefix):
    user_data = []
    # Sample the number of series the user is interested in
    num_series = random.randint(4, 7)
    for _ in range(num_series):
        series = df_npo.sample().to_dict(orient='records')[0]  # Select a random row from the DataFrame
        genre1 = series.get('Genre_1', '')
        genre2 = series.get('Genre_2', '')
        broadcaster = series.get('Broadcaster', '')
        if genre1 in preferences and genre2 in preferences and broadcaster in preferences.get('Broadcasters', {}):
            series['genre1_weight'] *= preferences[genre1]
            series['genre2_weight'] *= preferences[genre2]
            series['political_bias_bc'] *= preferences['Broadcasters'].get(broadcaster, 1.0)
            series['user_id'] = f"{user_id_prefix}"  # Generate unique user ID
            series['persona'] = persona
            user_data.append(series)
    return user_data

# Generate synthesized user data for each persona and concatenate into a single DataFrame
all_user_dfs = []

for persona, preferences in personas.items():
    persona_users = []
    for i in range(1, num_users_per_persona + 1):
        user_data = synthesize_user_data(persona, preferences, df_npo, f"{persona}_user_{i}")
        persona_df = pd.DataFrame(user_data)
        persona_users.append(persona_df)
    all_user_dfs.extend(persona_users)

# Concatenate all user DataFrames into a single DataFrame
final_user_df = pd.concat(all_user_dfs, ignore_index=True)

In [36]:
# Reorganize columns to make user_id the first column
cols = final_user_df.columns.tolist()
cols = ['user_id'] + [col for col in cols if col != 'user_id']
df_user = final_user_df[cols]
df_user

,user_id,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,genre1_weight,genre2_weight,political_bias_bc,polarization_score,inverted_polarization_score,Episode,persona
0,Ambitieuze Jongeren_user_1,548,State of Hate,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Reportage,BNNVARA,De bom in Oklahoma City,Timothy McVeigh pleegde een aanslag op een ove...,https://assets-start.npo.nl/resources/2024/01/...,State of Hate Informatief Reportage De bom in ...,0.019841,0.019841,-0.525,-5.46875,0.453125,3,Ambitieuze Jongeren
1,Ambitieuze Jongeren_user_1,1721,De verschrikkelijke jaren tachtig,https://assets-start.npo.nl/resources/2023/06/...,Serie,Drama,VPRO,Ivo,In de woongroep is veel aan de hand: Karin den...,https://assets-start.npo.nl/resources/2023/06/...,De verschrikkelijke jaren tachtig Serie Drama ...,0.015873,0.000000,-0.300,-0.00000,1.000000,4,Ambitieuze Jongeren
2,Ambitieuze Jongeren_user_1,660,Indonesia roept!,https://assets-start.npo.nl/resources/2023/07/...,Informatief,Geschiedenis,NTR,Omgekeerde wereld,Als de Duitsers in mei 1940 Nederland binnenva...,https://assets-start.npo.nl/resources/2023/09/...,Indonesia roept! Informatief Geschiedenis Omge...,0.019841,0.019841,-0.075,-2.34375,0.765625,4,Ambitieuze Jongeren
3,Ambitieuze Jongeren_user_1,242,De Avondshow met Arjen Lubach,https://assets-start.npo.nl/resources/2024/01/...,Amusement,Komisch/Satire,VPRO,"Bestuurscrisis in Almere, Promes",Via de waan van de dag duikt Arjen Lubach in e...,https://assets-start.npo.nl/resources/2024/03/...,De Avondshow met Arjen Lubach Amusement Komisc...,0.047619,0.083333,-0.300,-3.00000,0.700000,29,Ambitieuze Jongeren
4,Ambitieuze Jongeren_user_2,362,Boer zoekt vrouw,https://assets-start.npo.nl/resources/2023/09/...,Human Interest,Liefde,KRO-NCRV,Ook Claudia en Bernice maken hun definitieve k...,Hun eerste reisje samen brengt Richard en Robi...,https://assets-start.npo.nl/resources/2023/12/...,Boer zoekt vrouw Human Interest Liefde Ook Cla...,0.011905,0.000000,-0.100,-0.00000,1.000000,11,Ambitieuze Jongeren
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4544,Behoedzame Senioren_user_125,1672,MORTEN,https://assets-start.npo.nl/resources/2023/06/...,Serie,Spanning,AVROTROS,Esther,Morten Mathijsen gaat als formateur van een ni...,https://assets-start.npo.nl/resources/2023/07/...,MORTEN Serie Spanning Esther Morten Mathijsen ...,0.015625,0.000000,-0.050,-0.00000,1.000000,6,Behoedzame Senioren
4545,Behoedzame Senioren_user_125,1749,Tropenjaren,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,BNNVARA,De dochter van de baas,Rosa wil haar plekje veroveren in het old boys...,https://assets-start.npo.nl/resources/2023/09/...,Tropenjaren Serie Drama De dochter van de baas...,0.015625,0.000000,-0.175,-0.00000,1.000000,8,Behoedzame Senioren
4546,Behoedzame Senioren_user_125,493,Kiespijn,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Spel/quiz,NTR,Kiespijn: Tweede Kamerverkiezingen,De politieke panelquiz wordt gespeeld met twee...,https://assets-start.npo.nl/resources/2023/11/...,Kiespijn Informatief Spel/quiz Kiespijn: Tweed...,0.039062,0.000000,-0.075,-0.00000,1.000000,4,Behoedzame Senioren
4547,Behoedzame Senioren_user_125,640,Goedemorgen Nederland,https://assets-start.npo.nl/resources/2024/01/...,Informatief,Nieuws/actualiteiten,WNL,Goedemorgen Nederland,Te gast zijn hoofdredacteur EW Magazine Hella ...,https://assets-start.npo.nl/resources/2024/03/...,Goedemorgen Nederland Informatief Nieuws/actua...,0.039062,0.082031,0.525,6.56250,0.343750,51,Behoedzame Senioren


## USER POLITICAL BIAS

In [37]:
# add column to user dataset with watch history per user
user_ids = df_user['user_id'].unique()
watch_history = {'user_id': [], 'watch_history': []}
for user_id in user_ids:
    watched_videos = df_user.loc[df_user['user_id'] == user_id, 'episode_id'].tolist()
    watch_history['user_id'].append(user_id)
    watch_history['watch_history'].append(watched_videos)

watch_history_df = pd.DataFrame(watch_history)
df_user = pd.merge(df_user, watch_history_df, on='user_id')

In [38]:
# calculate and add political polarization rating of each user
user_ids = df_user['user_id'].unique()
for user_id in user_ids:
    episode_ids = df_user.loc[df_user['user_id'] == user_id, 'episode_id'].tolist()
    user_polarization_scores = df_npo.loc[df_npo['episode_id'].isin(episode_ids), 'polarization_score'].tolist()
    user_political_score = np.mean(user_polarization_scores)
    df_user.loc[df_user['user_id'] == user_id, 'political_rating'] = user_political_score

## SIMILARITY FUNCTIONS

In [39]:
# create function that gets similarity scores for a specific video
def get_similarities(episode_id):
    similarities = sim_df.loc[episode_id]
    return similarities.rename('similarity').to_frame()

# create function that adds similarity scores to complete dataframe
def recommend_videos(episode_id):
    similarities = get_similarities(episode_id)
    # drop similarity with the same article
    npo_df = df_npo.drop(episode_id, axis='rows').join(similarities)
    return npo_df.sort_values('similarity', ascending=False)


## ACCOUNTABILITY FUNCTION

In [40]:
# create filter function that removes recommendations that are politically polarizing to user
def filter_higher(user_rating, recommendations):
    # if user has a right political score
    if user_rating > 1:
        filtered_df = recommendations[recommendations['polarization_score'] <= user_rating + 1]
    # if user has a left political score
    elif user_rating < -1:
        filtered_df = recommendations[recommendations['polarization_score'] >= user_rating - 1]
    # if user has centre political score
    else:
        filtered_df = recommendations
    return filtered_df

# function that removes politically distant recommendations to user
def filter_buffer(user_rating, recommendations):
    filtered_df = recommendations[abs(recommendations['polarization_score'] - user_rating) < 5]
    return filtered_df

# function that adds polarization score and accountability score to dataframe
def accountability_score(df_recommendations):
    # accountability score of a recommendation is weighted score of similarity and polarization
    df_recommendations['accountability_score'] = df_recommendations['similarity'] * df_recommendations['inverted_polarization_score']
    return df_recommendations

# operationalization of accountability
def get_accountable_recommendations(df_user, user_id, df_recommendations):
    # get user information of specific user
    user_info = df_user[df_user['user_id'] == user_id]

    # get average political score of user
    user_rating = user_info.iloc[0]['political_rating']

    # filter recommendations so they are not politically polarizing for the user
    filtered_recs = filter_higher(user_rating, df_recommendations)

    # filter recommendatiosn so they are in range of political spectrum of user
    filtered_recs = filter_buffer(user_rating, filtered_recs)

    # create accountability score for all remaining recommendations
    accountable_recommendations = accountability_score(filtered_recs)

    # sort articles
    accountable_recommendations = accountable_recommendations.sort_values('accountability_score', ascending=False)

    return accountable_recommendations



## CREATE ACCOUNTABLE RECOMMENDATIONS (EXAMPLE C-S)

In [41]:
# get random user_id
random_user = df_user.sample(1)
user_id = random_user.iloc[0]['user_id']
# get recommendations of (random) last watched video
random_video = df_npo.sample(1)
video_id = random_video.iloc[0]['episode_id']
df_recommendations = recommend_videos(video_id)
# get accountable recommendations
df_recommendations = get_accountable_recommendations(df_user, user_id, df_recommendations)

<ipython-input-40-93845e4a5483>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommendations['accountability_score'] = df_recommendations['similarity'] * df_recommendations['inverted_polarization_score']


## SERENDIPITY

In [42]:
# operationalization of serendipity
# serendipity = relevance * unexpectedness
# relevance: all unseen shows
# unexpectedness: (1 - average similarity score for each video to the users history) * (0.5 if broadcaster is not in users preference + 0.5 if genre is not in users preference)
def get_recommendations_serendipity(df_recommendations, df_user, user_id, sim_df, personas):

    # get the user's watched videos
    watched_videos = df_user.loc[df_user['user_id'] == user_id, 'watch_history'].values[0]

    # measure average similarity score for unseen shows
    average_sim_df = pd.DataFrame()
    for i, watched_video in enumerate(watched_videos):
        similarities = sim_df.loc[watched_video]
        col = f'col{i}'
        average_sim_df[col] = similarities
    average_sim_df['average'] = average_sim_df.mean(axis=1)

    # calculate unexpectedness
    df_recommendations['unexpectedness'] = 1 - average_sim_df['average']

    # get list of expected genres
    persona = df_user.loc[df_user['user_id'] == user_id, 'persona'].values[0]
    persona = personas[persona]
    genres = set([value for key, value in persona.items() if key != 'Broadcasters'])
    sorted_values = sorted(genres)
    list_genres = [key for key, value in persona.items() if value in sorted_values[-2:]]

    # get list of expected broadcasters
    broadcasters = persona['Broadcasters']
    list_broadcasters = [key for key, value in broadcasters.items() if value >= 0.1]

    df_recommendations['preferences'] = 0
    df_recommendations.loc[~df_recommendations['Broadcaster'].isin(list_broadcasters), 'preferences'] += 0.5
    df_recommendations.loc[~df_recommendations['Genre_1'].isin(list_genres) | ~df_recommendations['Genre_2'].isin(list_genres), 'preferences'] += 0.5

    df_recommendations['unexpectedness'] = df_recommendations['unexpectedness'] * df_recommendations['preferences']

    # include relevance into dataframe
    df_recommendations['relevance'] = 1 * df_recommendations['similarity']
    df_recommendations.loc[df_recommendations['episode_id'].isin(watched_videos), 'relevance'] = 0

    # add serendipity to dataframe and remove irrelevant videos
    df_recommendations['serendipity'] = df_recommendations['unexpectedness'] * df_recommendations['relevance']
    df_recommendations = df_recommendations[df_recommendations['serendipity'] != 0]

    df_serendipity = df_recommendations.sort_values(by='serendipity', ascending=False)

    return df_serendipity


In [43]:
df_recommendations_serendipity = get_recommendations_serendipity(df_recommendations, df_user, user_id, sim_df, personas)
df_recommendations_serendipity.head(10)

,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,...,political_bias_bc,polarization_score,inverted_polarization_score,Episode,similarity,accountability_score,unexpectedness,preferences,relevance,serendipity
66,66,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Proosten op het politieke jaar,Begin dit jaar moest kabinet-Rutte IV vele cri...,https://assets-start.npo.nl/resources/2023/12/...,De Hofbar Informatief Politiek Proosten op het...,...,2.0,4.375,0.5625,15,0.179303,0.100858,0.990126,1.0,0.179303,0.177532
74,74,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,"Wopke Hoekstra, de nieuwe klimaatpaus?",Na twaalf jaar Nederlandse politiek gaat Wopke...,https://assets-start.npo.nl/resources/2023/10/...,"De Hofbar Informatief Politiek Wopke Hoekstra,...",...,2.0,4.375,0.5625,5,0.131429,0.073929,0.986755,1.0,0.131429,0.129688
75,75,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Toeslagenkinderen staan in de kou,Kinderen van toeslagenouders hebben bij DUO ge...,https://assets-start.npo.nl/resources/2023/09/...,De Hofbar Informatief Politiek Toeslagenkinder...,...,2.0,4.375,0.5625,4,0.113223,0.063688,0.985382,1.0,0.113223,0.111568
70,70,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Gemeente Amsterdam dramt windmolens door,Provincie Noord-Holland stemt tegen windmolens...,https://assets-start.npo.nl/resources/2023/11/...,De Hofbar Informatief Politiek Gemeente Amster...,...,2.0,4.375,0.5625,11,0.085471,0.048078,0.988967,1.0,0.085471,0.084528
57,57,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Amsterdamse windmolenwaanzin,Amsterdam heeft lak aan een besluit van de pro...,https://assets-start.npo.nl/resources/2024/03/...,De Hofbar Informatief Politiek Amsterdamse win...,...,2.0,4.375,0.5625,9,0.085730,0.048223,0.985049,1.0,0.085730,0.084448
68,68,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Wie ruimt Rutte's rotzooi op?,Na jaren regeren wil de VVD geen bewindsperson...,https://assets-start.npo.nl/resources/2023/12/...,De Hofbar Informatief Politiek Wie ruimt Rutte...,...,2.0,4.375,0.5625,13,0.081815,0.046021,0.989627,1.0,0.081815,0.080966
58,58,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Kankerpatiënten in de kou,Kankerpatiënten zijn de dupe van de financiële...,https://assets-start.npo.nl/resources/2024/03/...,De Hofbar Informatief Politiek Kankerpatiënten...,...,2.0,4.375,0.5625,8,0.081678,0.045944,0.989645,1.0,0.081678,0.080832
67,67,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Hoe steekt de vape-industrie in elkaar?,"Via het schoolplein of online, jongeren komen ...",https://assets-start.npo.nl/resources/2023/12/...,De Hofbar Informatief Politiek Hoe steekt de v...,...,2.0,4.375,0.5625,14,0.081055,0.045593,0.989724,1.0,0.081055,0.080222
209,209,Pointer,https://assets-start.npo.nl/resources/2023/09/...,Informatief,Onderzoeksjournalistiek,KRO-NCRV,De giftige lobby: de macht van chemiebedrijven,Pointer onderzoekt de macht van de Brusselse p...,https://assets-start.npo.nl/resources/2024/01/...,Pointer Informatief Onderzoeksjournalistiek De...,...,-1.0,-1.875,0.8125,1,0.158391,0.128693,0.498312,0.5,0.158391,0.078928
69,69,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Was monsterzege PVV echt zo onverwacht?,De verkiezingen zijn geweest en de kiezer heef...,https://assets-start.npo.nl/resources/2023/11/...,De Hofbar Informatief Politiek Was monsterzege...,...,2.0,4.375,0.5625,12,0.079188,0.044543,0.984776,1.0,0.079188,0.077983


## FAIRNESS

In [44]:
# operationalization of fairness
def get_recommendations_fairness(df_recommendations, df_user, user_id, personas):

    # Get relevant broadcasters and genres
    persona = df_user.loc[df_user['user_id'] == user_id, 'persona'].values[0]
    persona = personas[persona]
    genres = set([value for key, value in persona.items() if key != 'Broadcasters'])
    sorted_values = sorted(genres)
    list_genres = [key for key, value in persona.items() if value in sorted_values[-2:]]

    broadcasters = persona['Broadcasters']
    list_broadcasters = [key for key, value in broadcasters.items() if value >= 0.1]

    # keep recommendations that are fair in regards to their original persona
    df_fair_recommendations = df_recommendations[df_recommendations['Broadcaster'].isin(list_broadcasters)
                                                 & (df_recommendations['Genre_1'].isin(list_genres)
                                                     | df_recommendations['Genre_2'].isin(list_genres))]

    # Check if there are any movies in the relevant genres
    if not df_fair_recommendations.empty:
        # Select top-ranked movies within relevant genres
        fair_recommendations = df_fair_recommendations.head(10)
    else:
      fair_recommendations = df_recommendations.head(10) #regular recommendation if no relevant genres are matched
    return fair_recommendations

In [45]:
df_recommendations_fairness = get_recommendations_fairness(df_recommendations, df_user, user_id, personas)
df_recommendations_fairness

,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,...,political_bias_bc,polarization_score,inverted_polarization_score,Episode,similarity,accountability_score,unexpectedness,preferences,relevance,serendipity
1318,1318,EenVandaag,https://images.poms.omroep.nl/image/1633886,Informatief,Nieuws/actualiteiten,AVROTROS,EenVandaag,Aantal Nederlanders met obesitas neemt snel to...,https://assets-start.npo.nl/resources/2024/03/...,EenVandaag Informatief Nieuws/actualiteiten Aa...,...,-0.5,-0.9375,0.90625,54,0.057833,0.052411,0.479978,0.5,0.057833,0.027759
1218,1218,Nieuwsuur,https://assets-start.npo.nl/resources/2023/11/...,Informatief,Nieuws/actualiteiten,NOS,Nieuwsuur,'Onderzeeërs bestellen bij Fransen is gemiste ...,https://assets-start.npo.nl/resources/2024/03/...,Nieuwsuur Informatief Nieuws/actualiteiten 'On...,...,0.0,0.0000,1.00000,72,0.042051,0.042051,0.478295,0.5,0.042051,0.020113
153,153,Op1,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Nieuws/actualiteiten,MAX,Op1 - 22 februari 2024,Nynke Nijman en Jens van Tricht over een nieuw...,https://assets-start.npo.nl/resources/2024/02/...,Op1 Informatief Nieuws/actualiteiten Op1 - 22 ...,...,1.0,1.8750,0.81250,34,0.045548,0.037008,0.482139,0.5,0.045548,0.021960
1311,1311,EenVandaag,https://images.poms.omroep.nl/image/1633886,Informatief,Nieuws/actualiteiten,AVROTROS,EenVandaag,Een op de tien jongeren heeft weleens naaktfot...,https://assets-start.npo.nl/resources/2024/03/...,EenVandaag Informatief Nieuws/actualiteiten Ee...,...,-0.5,-0.9375,0.90625,61,0.040743,0.036923,0.483311,0.5,0.040743,0.019691
1322,1322,EenVandaag,https://images.poms.omroep.nl/image/1633886,Informatief,Nieuws/actualiteiten,AVROTROS,EenVandaag,Onderzeeboten voor Defensie mogelijk niet door...,https://assets-start.npo.nl/resources/2024/02/...,EenVandaag Informatief Nieuws/actualiteiten On...,...,-0.5,-0.9375,0.90625,50,0.039749,0.036023,0.479392,0.5,0.039749,0.019055
1873,1873,Ten minste houdbaar tot,https://images.poms.omroep.nl/image/1711996,Serie,Komisch,AVROTROS,De 20-secondenregel,Als blijkt dat Mila liever bij haar vriendinne...,https://assets-start.npo.nl/resources/2023/07/...,Ten minste houdbaar tot Serie Komisch De 20-se...,...,-0.5,-0.0000,1.00000,7,0.033879,0.033879,0.499493,0.5,0.033879,0.016922
617,617,Verborgen Werelden | Nieuwsuur,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Nieuws/actualiteiten,NOS,Nederlandse spil in handel naar Rusland | Nieu...,"Een Nederlands bedrijf, eigendom van een Rus, ...",https://assets-start.npo.nl/resources/2023/12/...,Verborgen Werelden | Nieuwsuur Informatief Nie...,...,0.0,0.0000,1.00000,2,0.033830,0.033830,0.485057,0.5,0.033830,0.016410
1324,1324,EenVandaag,https://images.poms.omroep.nl/image/1633886,Informatief,Nieuws/actualiteiten,AVROTROS,EenVandaag,Rapport Fraudebeleid overheid gepresenteerd * ...,https://assets-start.npo.nl/resources/2024/02/...,EenVandaag Informatief Nieuws/actualiteiten Ra...,...,-0.5,-0.9375,0.90625,48,0.036596,0.033165,0.481369,0.5,0.036596,0.017616
1824,1824,Jos,https://images.poms.omroep.nl/image/1711942,Serie,Komisch,AVROTROS,Yeh yeh,"Jos, een man met alleen maar goede bedoelingen...",https://images.poms.omroep.nl/image/1711956,"Jos Serie Komisch Yeh yeh Jos, een man met all...",...,-0.5,-0.0000,1.00000,1,0.032664,0.032664,0.499073,0.5,0.032664,0.016302
1811,1811,De wereld in 2100 | Nieuwsuur,https://assets-start.npo.nl/resources/2023/09/...,Informatief,Nieuws/actualiteiten,NOS,De macht van het getal,Waarom zijn Afrikaanse landen zo slecht verteg...,https://assets-start.npo.nl/resources/2023/09/...,De wereld in 2100 | Nieuwsuur Informatief Nieu...,...,0.0,0.0000,1.00000,4,0.031904,0.031904,0.483850,0.5,0.031904,0.015437


## EXAMPLE OF FINAL RECOMMENDATIONS

In [46]:
print(df_recommendations['episode_id'].head(10))
print(df_recommendations_serendipity['episode_id'].head(10))
print(df_recommendations_fairness['episode_id'].head(10))

209      209
66        66
74        74
1947    1947
1570    1570
718      718
75        75
372      372
1897    1897
1318    1318
Name: episode_id, dtype: int64
66      66
74      74
75      75
70      70
57      57
68      68
58      58
67      67
209    209
69      69
Name: episode_id, dtype: int64
1318    1318
1218    1218
153      153
1311    1311
1322    1322
1873    1873
617      617
1324    1324
1824    1824
1811    1811
Name: episode_id, dtype: int64





## Results and Analysis

In [47]:
user_id

'Behoedzame Senioren_user_48'

In [48]:
user_history = df_user[df_user['user_id'] == user_id]
user_history[['user_id','Broadcaster','Genre_1','Genre_2','episode_id','polarization_score','political_rating']]

,user_id,Broadcaster,Genre_1,Genre_2,episode_id,polarization_score,political_rating
4192,Behoedzame Senioren_user_48,NOS,Informatief,Nieuws/actualiteiten,1150,0.0000,0.6875
4193,Behoedzame Senioren_user_48,BNNVARA,Serie,Drama,1920,-0.0000,0.6875
4194,Behoedzame Senioren_user_48,WNL,Informatief,Nieuws/actualiteiten,625,6.5625,0.6875
4195,Behoedzame Senioren_user_48,NOS,Informatief,Nieuws/actualiteiten,1141,0.0000,0.6875
4196,Behoedzame Senioren_user_48,VPRO,Documentaire,Reizen,1203,-3.1250,0.6875


In [49]:
df_recommendations[['Broadcaster','Genre_1','Genre_2','episode_id','polarization_score']].head(5)

,Broadcaster,Genre_1,Genre_2,episode_id,polarization_score
209,KRO-NCRV,Informatief,Onderzoeksjournalistiek,209,-1.875
66,PowNed,Informatief,Politiek,66,4.375
74,PowNed,Informatief,Politiek,74,4.375
1947,AVROTROS,Serie,Drama,1947,-0.000
1570,BNNVARA,Serie,Drama,1570,-0.000


In [50]:
df_recommendations_fairness[['Broadcaster','Genre_1','Genre_2','episode_id','Description','polarization_score']].head(5)

,Broadcaster,Genre_1,Genre_2,episode_id,Description,polarization_score
1318,AVROTROS,Informatief,Nieuws/actualiteiten,1318,Aantal Nederlanders met obesitas neemt snel to...,-0.9375
1218,NOS,Informatief,Nieuws/actualiteiten,1218,'Onderzeeërs bestellen bij Fransen is gemiste ...,0.0000
153,MAX,Informatief,Nieuws/actualiteiten,153,Nynke Nijman en Jens van Tricht over een nieuw...,1.8750
1311,AVROTROS,Informatief,Nieuws/actualiteiten,1311,Een op de tien jongeren heeft weleens naaktfot...,-0.9375
1322,AVROTROS,Informatief,Nieuws/actualiteiten,1322,Onderzeeboten voor Defensie mogelijk niet door...,-0.9375


In [51]:
df_recommendations_serendipity[['Broadcaster','Genre_1','Genre_2','episode_id','Description','polarization_score']].head(5)

,Broadcaster,Genre_1,Genre_2,episode_id,Description,polarization_score
66,PowNed,Informatief,Politiek,66,Begin dit jaar moest kabinet-Rutte IV vele cri...,4.375
74,PowNed,Informatief,Politiek,74,Na twaalf jaar Nederlandse politiek gaat Wopke...,4.375
75,PowNed,Informatief,Politiek,75,Kinderen van toeslagenouders hebben bij DUO ge...,4.375
70,PowNed,Informatief,Politiek,70,Provincie Noord-Holland stemt tegen windmolens...,4.375
57,PowNed,Informatief,Politiek,57,Amsterdam heeft lak aan een besluit van de pro...,4.375


## EXTRA: USER SIMILARITY

In [52]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

df_user_sm = df_user[['user_id', 'episode_id']]

# Compute user-item matrix
user_item_matrix = df_user_sm.pivot_table(index='user_id', columns='episode_id', aggfunc=lambda x: 1, fill_value=0)

# Normalize user-item matrix
scaler = MinMaxScaler()
normalized_matrix = scaler.fit_transform(user_item_matrix)

# Compute similarity matrix
similarity_matrix = cosine_similarity(normalized_matrix)

def get_similar_users(user_id, k=10):
    user_index = user_item_matrix.index.get_loc(user_id)
    similar_users = similarity_matrix[user_index]
    similar_users_indices = similar_users.argsort()[-k-1:-1][::-1]
    similar_users_similarities = similar_users[similar_users_indices]
    return similar_users_indices, similar_users_similarities

similar_users_indices, similar_users_similarities = get_similar_users(user_id)

similar_users = user_item_matrix.iloc[similar_users_indices]

watched_episodes = []
episode_user_sim = {}
for (index, row), sim in zip(similar_users.iterrows(), similar_users_similarities):
    # List to store column names where value is 1 for current row
    watched_episodes_user = []
    # Loop through columns
    for column in similar_users.columns:
        # Check if value is 1
        if row[column] == 1:
            watched_episodes.append(column)
            if column in episode_user_sim:
                if sim > episode_user_sim[column]:
                    episode_user_sim[column] = sim
            else:
                episode_user_sim[column] = sim

watched_episodes = list(set(watched_episodes))

user_watched_episodes = df_user.loc[df_user['user_id'] == user_id, 'watch_history'].values[0]

user_not_watched_videos = [x for x in watched_episodes if x not in user_watched_episodes]

df_user_recommendations = df_npo[df_npo['episode_id'].isin(user_not_watched_videos)]
df_user_recommendations['similarity'] = df_user_recommendations['episode_id'].map(episode_user_sim)
df_user_recommendations

<ipython-input-52-3853167b198a>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_recommendations['similarity'] = df_user_recommendations['episode_id'].map(episode_user_sim)


,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,genre1_weight,genre2_weight,political_bias_bc,polarization_score,inverted_polarization_score,Episode,similarity
54,54,Een porseleinen huwelijk,https://assets-start.npo.nl/resources/2023/06/...,Documentaire,Geschiedenis,BNNVARA,Een porseleinen huwelijk,Terwijl de regering in alle stilte inzet op ee...,https://assets-start.npo.nl/resources/2023/07/...,Een porseleinen huwelijk Documentaire Geschied...,1.25,1.25,-3.5,-5.46875,0.453125,2,0.200000
64,64,De Hofbar,https://assets-start.npo.nl/resources/2023/12/...,Informatief,Politiek,PowNed,Vape-industrie heeft lak aan overheid,"De vape-industrie negeert het smaakjesverbod, ...",https://assets-start.npo.nl/resources/2024/01/...,De Hofbar Informatief Politiek Vape-industrie ...,1.25,1.75,2.0,4.37500,0.562500,2,0.223607
118,118,Radar,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Consumenten informatie,AVROTROS,Radar,Terwijl de Eerste Kamer in debat is over de mo...,https://assets-start.npo.nl/resources/2024/01/...,Radar Informatief Consumenten informatie Terwi...,1.25,1.25,-0.5,-0.78125,0.921875,2,0.200000
145,145,Op1,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Nieuws/actualiteiten,MAX,Op1 - 5 maart 2024,"Raymond Mens, Roderick Veelo en Sofie van den ...",https://assets-start.npo.nl/resources/2024/03/...,Op1 Informatief Nieuws/actualiteiten Op1 - 5 m...,1.25,1.50,1.0,1.87500,0.812500,42,0.223607
185,185,Gevlucht na de val,https://assets-start.npo.nl/resources/2023/09/...,Human Interest,Biografie,BNNVARA,Mahbob Darabi,Mahbob Darabi had jarenlang twee populaire sno...,https://assets-start.npo.nl/resources/2023/09/...,Gevlucht na de val Human Interest Biografie Ma...,0.75,1.00,-3.5,-2.62500,0.737500,3,0.200000
194,194,Rutger en de Nationalisten,https://assets-start.npo.nl/resources/2023/09/...,Documentaire,Human Interest,PowNed,Grenzen,Behalve Nederlands nationalisme is er ook nati...,https://assets-start.npo.nl/resources/2023/06/...,Rutger en de Nationalisten Documentaire Human ...,1.25,1.25,2.0,3.12500,0.687500,6,0.200000
216,216,Stand van Nederland,https://assets-start.npo.nl/resources/2024/01/...,Informatief,Onderzoeksjournalistiek,WNL,Overnames bij familiebedrijven,Meer dan de helft van de bedrijven wordt gerun...,https://assets-start.npo.nl/resources/2024/01/...,Stand van Nederland Informatief Onderzoeksjour...,1.25,1.50,3.5,6.56250,0.343750,4,0.200000
228,228,Ryan is zwanger,https://assets-start.npo.nl/resources/2023/06/...,Human Interest,Levenswijze,BNNVARA,Ryan is zwanger,"Als je Ryan en David, allebei met volle baard,...",https://assets-start.npo.nl/resources/2023/07/...,Ryan is zwanger Human Interest Levenswijze Als...,0.75,1.25,-3.5,-3.28125,0.671875,1,0.223607
258,258,De Avondshow met Arjen Lubach,https://assets-start.npo.nl/resources/2024/01/...,Amusement,Komisch/Satire,VPRO,"Rapport van Plasterk, Taylor Swift",Het rapport van informateur Ronald Plasterk ve...,https://assets-start.npo.nl/resources/2024/02/...,De Avondshow met Arjen Lubach Amusement Komisc...,1.00,1.50,-2.0,-3.00000,0.700000,13,0.223607
266,266,De Avondshow met Arjen Lubach,https://assets-start.npo.nl/resources/2024/01/...,Amusement,Komisch/Satire,VPRO,"Tijdcapsules, VVD-congres","De VVD kwam bijeen in Noordwijkerhout, maar wa...",https://assets-start.npo.nl/resources/2024/01/...,De Avondshow met Arjen Lubach Amusement Komisc...,1.00,1.50,-2.0,-3.00000,0.700000,5,0.200000


## CREATE ACCOUNTABLE RECOMMENDATIONS (EXAMPLE U-S)

In [53]:
df_user_recommendations = get_accountable_recommendations(df_user, user_id, df_user_recommendations)
df_user_recommendations

<ipython-input-40-93845e4a5483>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommendations['accountability_score'] = df_recommendations['similarity'] * df_recommendations['inverted_polarization_score']


,episode_id,Serie,Image_serie,Genre_1,Genre_2,Broadcaster,Title,Description,Image_ep,Content,genre1_weight,genre2_weight,political_bias_bc,polarization_score,inverted_polarization_score,Episode,similarity,accountability_score
1574,1574,The Spectacular,https://assets-start.npo.nl/resources/2023/06/...,Serie,Drama,VPRO,De duivel zelf,Een brute aanslag door de IRA-cel in Limburg w...,https://assets-start.npo.nl/resources/2023/07/...,The Spectacular Serie Drama De duivel zelf Een...,0.50,0.00,-2.0,-0.00000,1.000000,3,0.258199,0.258199
1890,1890,Santos,https://assets-start.npo.nl/resources/2023/09/...,Serie,Drama,BNNVARA,Loyaliteit,Glenn en Jermaine moeten een klus doen voor Ki...,https://assets-start.npo.nl/resources/2023/10/...,Santos Serie Drama Loyaliteit Glenn en Jermain...,0.50,0.00,-3.5,-0.00000,1.000000,6,0.223607,0.223607
843,843,Dit Was Het Nieuws,https://assets-start.npo.nl/resources/2024/02/...,Amusement,Komisch/Satire,AVROTROS,Dit Was Het Nieuws,Te gast zijn Lisa Loeb en Rayen Panday.,https://assets-start.npo.nl/resources/2023/09/...,Dit Was Het Nieuws Amusement Komisch/Satire Te...,1.00,1.50,-0.5,-0.75000,0.925000,4,0.223607,0.206836
1086,1086,De broeders Van Rossem,https://assets-start.npo.nl/resources/2023/09/...,Informatief,Kunst/cultuur,NTR,Den Haag en de Koude Oorlog,Breekt met het agressieve Russische buitenland...,https://assets-start.npo.nl/resources/2023/09/...,De broeders Van Rossem Informatief Kunst/cultu...,1.25,0.50,-1.5,-0.93750,0.906250,3,0.223607,0.202644
1771,1771,Voetbalmaffia,https://assets-start.npo.nl/resources/2023/06/...,Serie,Drama,KRO-NCRV,Dubbelspel,Johnny denkt al zijn problemen te hebben opgel...,https://assets-start.npo.nl/resources/2023/07/...,Voetbalmaffia Serie Drama Dubbelspel Johnny de...,0.50,0.00,-1.0,-0.00000,1.000000,5,0.200000,0.200000
1007,1007,NOS Journaal 20.00 uur,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Nieuws/actualiteiten,NOS,NOS Journaal,"Met het laatste nieuws, gebeurtenissen van nat...",https://assets-start.npo.nl/resources/2024/03/...,NOS Journaal 20.00 uur Informatief Nieuws/actu...,1.25,1.50,0.0,0.00000,1.000000,45,0.200000,0.200000
1259,1259,De Joodse Raad,https://assets-start.npo.nl/resources/2024/03/...,Serie,Drama,EO,De gele vlek,De anti-Joodse maatregelen van de Duitsers wor...,https://assets-start.npo.nl/resources/2024/03/...,De Joodse Raad Serie Drama De gele vlek De ant...,0.50,0.00,2.5,0.00000,1.000000,2,0.200000,0.200000
1426,1426,De wereld rond met boeken,https://assets-start.npo.nl/resources/2023/12/...,Documentaire,Kunst/cultuur,MAX,Spanje (2/2),Richard E. Grant vervolgt zijn reis door Andal...,https://assets-start.npo.nl/resources/2024/02/...,De wereld rond met boeken Documentaire Kunst/c...,1.25,0.50,1.0,0.62500,0.937500,6,0.200000,0.187500
118,118,Radar,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Consumenten informatie,AVROTROS,Radar,Terwijl de Eerste Kamer in debat is over de mo...,https://assets-start.npo.nl/resources/2024/01/...,Radar Informatief Consumenten informatie Terwi...,1.25,1.25,-0.5,-0.78125,0.921875,2,0.200000,0.184375
145,145,Op1,https://assets-start.npo.nl/resources/2023/06/...,Informatief,Nieuws/actualiteiten,MAX,Op1 - 5 maart 2024,"Raymond Mens, Roderick Veelo en Sofie van den ...",https://assets-start.npo.nl/resources/2024/03/...,Op1 Informatief Nieuws/actualiteiten Op1 - 5 m...,1.25,1.50,1.0,1.87500,0.812500,42,0.223607,0.181681
